# Paradigma de posição baseado em grade

Esse paradigma estabelece as posições que caixas podem sem colocadas a priori, dentro de uma grade, que servem como coordenadas da posição da caixa.


In [8]:
import gurobipy as gp
from gurobipy import GRB

# Parâmetros do problema (exemplo)
L, W, H = 12, 8, 8  # dimensões do contêiner

# Tipos de caixas: (li, wi, hi) e quantidade disponível bi, escolhei vi=1 para maximar a quantidade de caixas no container
boxes = [
    (6, 3, 2, 2),
    (6, 4, 3, 5),
    (8, 3, 2, 3),
    (4, 3, 2, 2),
    (4, 4, 3, 3)
]

# Calcular o valor v_i como o volume relativo de cada caixa
v = [ (l * w * h) / (L * W * H) for (l, w, h, bi) in boxes ]

m = len(boxes)

# Conjuntos possíveis de posições, seguindo os "normal patterns" (simplificação)
# para cada dimensão e cada tipo de caixa
X = [range(L - li + 1) for (li, _, _, _) in boxes]
Y = [range(W - wi + 1) for (_, wi, _, _) in boxes]
Z = [range(H - hi + 1) for (_, _, hi, _) in boxes]

# Criar modelo
model = gp.Model("GridBasedPosition")

# Variáveis binárias x[i,p,q,r]: 1 se a caixa do tipo i está na posição (p,q,r)
x = {}
for i in range(m):
    li, wi, hi, bi = boxes[i]
    for p in X[i]:
        for q in Y[i]:
            for r in Z[i]:
                x[i, p, q, r] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{p}_{q}_{r}")

model.update()

# Função objetivo: objetivo depende de vi
model.setObjective(
    gp.quicksum(
        v[i] * x[i, p, q, r]
        for i in range(m)
        for p in X[i]
        for q in Y[i]
        for r in Z[i]
    ), 
    GRB.MAXIMIZE
)

# Restrição 1: Não sobrepor caixas
# Para cada posição no contêiner, só pode haver no máximo 1 caixa cobrindo essa posição

for xp in range(L):
    for yq in range(W):
        for zr in range(H):
            # Soma das variáveis que cobrem o ponto (xp,yq,zr) ≤ 1
            covering_vars = []
            for i in range(m):
                li, wi, hi, bi = boxes[i]
                for p in X[i]:
                    for q in Y[i]:
                        for r in Z[i]:
                            if p <= xp < p + li and q <= yq < q + wi and r <= zr < r + hi:
                                covering_vars.append(x[i, p, q, r])
            if covering_vars:
                model.addConstr(gp.quicksum(covering_vars) <= 1)

# Restrição 2: Quantidade máxima de cada tipo de caixa
for i in range(m):
    li, wi, hi, bi = boxes[i]
    model.addConstr(
        gp.quicksum(x[i, p, q, r] for p in X[i] for q in Y[i] for r in Z[i]) <= bi
    )

model.optimize()

# Mostrar solução (posições das caixas carregadas)
with open('dados_gridbased.txt','w') as f:
    f.write(f"{L} {W} {H}\n")  # primeira linha: dimensões do contêiner
    for i in range(m):
        li, wi, hi, bi = boxes[i]
        for p in X[i]:
            for q in Y[i]:
                for r in Z[i]:
                    if x[i, p, q, r].X > 0.5:
                        print(f"Caixa tipo {i} colocada na posição ({p}, {q}, {r})")
                        f.write(f"{p} {q} {r} {li} {wi} {hi} {i} {1}\n")



Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 773 rows, 1362 columns and 59178 nonzeros
Model fingerprint: 0x2746032b
Variable types: 0 continuous, 1362 integer (1362 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e-02, 9e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 0.8125000
Presolve removed 720 rows and 1252 columns
Presolve time: 0.02s
Presolved: 53 rows, 110 columns, 494 nonzeros
Found heuristic solution: objective 0.4375000
Variable types: 0 continuous, 110 integer (110 binary)

Root relaxation: objective 9.140625e-01, 154 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0   

In [ ]:
import gurobipy as gp
from gurobipy import GRB
from itertools import product

def solve_grid_based_model():
    """
    Implements and solves the 3D container loading problem using the
    grid-based paradigm from the Fasano PDF [cite: 190-235].
    
    Includes the "normal patterns" optimization to reduce the grid size [cite: 238-244].
    """
    try:
        # --- 1. Model Data ---
        L, W, H = 12, 8, 8

        # Box data is now by TYPE: (dimensions), num_available
        box_types = {
            0: {'dims': (6, 3, 2), 'b': 2, 'val': (6*3*2)}, # Type 0 (Original boxes 1-2)
            1: {'dims': (6, 4, 3), 'b': 5, 'val': (6*4*3)}, # Type 1 (Original boxes 3-7)
            2: {'dims': (8, 3, 2), 'b': 3, 'val': (8*3*2)}, # Type 2 (Original boxes 8-10)
            3: {'dims': (4, 3, 2), 'b': 2, 'val': (4*3*2)}, # Type 3 (Original boxes 11-12)
            4: {'dims': (4, 4, 3), 'b': 3, 'val': (4*4*3)}, # Type 4 (Original boxes 13-15)
        }
        m = len(box_types)

        # --- 2. OPTIMIZATION: Generate "Normal Pattern" Coordinates ---
        def get_normal_pattern_coords(max_dim, all_dims):
            coords = {0}
            for d in sorted(list(all_dims)):
                new_coords = list(coords)
                for c in coords:
                    for k in range(1, (max_dim // d) + 1):
                        new_c = c + k * d
                        if new_c <= max_dim:
                            new_coords.append(new_c)
                        else:
                            break
                coords.update(new_coords)
            
            # Filter coords that are too large for any box to fit
            min_box_dim = min(all_dims)
            return sorted([c for c in coords if c <= max_dim - min_box_dim])

        all_lengths = {bt['dims'][0] for bt in box_types.values()}
        all_widths = {bt['dims'][1] for bt in box_types.values()}
        all_heights = {bt['dims'][2] for bt in box_types.values()}

        X_coords = get_normal_pattern_coords(L, all_lengths)
        Y_coords = get_normal_pattern_coords(W, all_widths)
        Z_coords = get_normal_pattern_coords(H, all_heights)

        # --- 3. Model Setup ---
        model = gp.Model("GridBasedContainerLoading")
        
        # --- 4. Decision Variables ---
        # x[i, p, q, r] is 1 if box type i is placed at (p,q,r)
        x = {}
        for i in range(m):
            li, wi, hi = box_types[i]['dims']
            # Only create variables for valid placements
            for p in [c for c in X_coords if c <= L - li]:
                for q in [c for c in Y_coords if c <= W - wi]:
                    for r in [c for c in Z_coords if c <= H - hi]:
                        x[i, p, q, r] = model.addVar(vtype=GRB.BINARY, name=f"x_{i}_{p}_{q}_{r}")

        # --- 5. Objective Function (12.18) ---
        model.setObjective(gp.quicksum(box_types[i]['val'] * x[i, p, q, r] 
                                    for i, p, q, r in x), GRB.MAXIMIZE)

        # --- 6. Constraints ---
        
        # A. Non-Overlapping Constraints (12.19)
        # For each point (s,t,u), at most one box can cover it
        for s, t, u in product(X_coords, Y_coords, Z_coords):
            boxes_covering_stu = []
            for i, p, q, r in x:
                li, wi, hi = box_types[i]['dims']
                if (p <= s < p + li) and (q <= t < q + wi) and (r <= u < r + hi):
                    boxes_covering_stu.append(x[i, p, q, r])
            
            if boxes_covering_stu:
                model.addConstr(gp.quicksum(boxes_covering_stu) <= 1, f"overlap_{s}_{t}_{u}")

        # B. Box Availability Constraints (12.20)
        for i in range(m):
            placements_of_type_i = gp.quicksum(x[i, p, q, r] for i_p, p, q, r in x if i_p == i)
            model.addConstr(placements_of_type_i <= box_types[i]['b'], f"avail_{i}")

        # --- 7. Solve ---
        model.optimize()

        # --- 8. Display Results ---
        if model.status == GRB.OPTIMAL:
            print("\n" + "="*40)
            print("OPTIMAL SOLUTION FOUND")
            print("="*40)
            
            packed_volume = model.ObjVal
            print(f"Total volume of packed boxes: {packed_volume}")
            print(f"Container volume utilization: { (packed_volume / (L*W*H)) * 100:.2f}%\n")

            print("Packed boxes layout:")
            print("-" * 30)
            for i, p, q, r in x:
                if x[i,p,q,r].X > 0.5:
                    print(f"  Box Type {i}: Dims {str(box_types[i]['dims']):<10} | Placed at (x={p}, y={q}, z={r})")
        else:
            print(f"Optimization ended with status: {model.status}")

    except gp.GurobiError as e:
        print(f"Gurobi Error code {e.errno}: {e}")

# Run the grid-based model
solve_grid_based_model()

with open('dados_gridbased_new.txt','w') as f:
    f.write(f"{L} {W} {H}\n")  # primeira linha: dimensões do contêiner
    for i in range(m):
        li, wi, hi, bi = boxes[i]
        for p in X[i]:
            for q in Y[i]:
                for r in Z[i]:
                    if x[i, p, q, r].X > 0.5:
                        print(f"Caixa tipo {i} colocada na posição ({p}, {q}, {r})")
                        f.write(f"{p} {q} {r} {li} {wi} {hi} {i} {1}\n")


Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 77 rows, 267 columns and 1872 nonzeros
Model fingerprint: 0x77ea3401
Variable types: 0 continuous, 267 integer (267 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+01, 7e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 624.0000000
Presolve removed 24 rows and 157 columns
Presolve time: 0.00s
Presolved: 53 rows, 110 columns, 494 nonzeros
Found heuristic solution: objective 384.0000000
Variable types: 0 continuous, 110 integer (110 binary)

Root relaxation: objective 7.020000e+02, 154 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0